In [1]:
import pyspark
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder
         .master("local[4]")
         .appName("test")
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 06:42:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget -nc http://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

File ‘taxi+_zone_lookup.csv’ already there; not retrieving.



In [6]:
df_zones = spark.read.csv("taxi+_zone_lookup.csv", header=True, inferSchema=True)

In [7]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [8]:
df_zones.write.parquet('zones', mode='overwrite')

# Read FHVHV data

In [9]:
from pyspark import SparkFiles

file_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz'
spark.sparkContext.addFile(file_url)

df_fhvhv = spark.read.csv(SparkFiles.get('fhvhv_tripdata_2021-01.csv.gz'), inferSchema=True, header=True)

In [10]:
df_fhvhv.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [11]:
df_fhvhv.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [12]:
df_fhvhv.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

## Infer schema by reading first 100 rows into pandas and then converting to spark making small adjustments

In [13]:
import pandas as pd

# Read the first 100 rows using Pandas
first_100_rows = pd.read_csv(file_url, compression='gzip', nrows=100, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [14]:
first_100_rows.dtypes

hvfhs_license_num               object
dispatching_base_num            object
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
SR_Flag                        float64
dtype: object

In [15]:
from pyspark.sql.types import IntegerType

In [16]:
first_100_rows_sp = spark.createDataFrame(first_100_rows)
first_100_rows_sp = (first_100_rows_sp
                     .withColumn("SR_Flag", first_100_rows_sp["SR_Flag"].cast(IntegerType()))
                     .withColumn("DOLocationID", first_100_rows_sp["DOLocationID"].cast(IntegerType()))
                     .withColumn("PULocationID", first_100_rows_sp["PULocationID"].cast(IntegerType())))
first_100_rows_sp.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

## Read FHVHV data again providing the schema

In [17]:
df_fhvhv = spark.read.csv(SparkFiles.get('fhvhv_tripdata_2021-01.csv.gz'), header=True, schema=first_100_rows_sp.schema)

In [18]:
df_fhvhv.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

## Write the obtained csv to partitioned parquet

In [20]:
df_fhvhv.repartition(24).write.parquet('fhvhv/2021/01/', mode="overwrite")

We can see that resulting parquet files are considerably smaller than the original csv

In [21]:
!ls -lh fhvhv/2021/01/ | head

total 461272
-rw-r--r--@ 1 alexlitvinov  staff     0B Mar  6 07:08 _SUCCESS
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00000-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00001-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00002-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00003-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00004-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00005-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00006-3701bd1c-87e1-4837-b98f-d44d1e7a0372-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Mar  6 07:08 part-00007-3701b

## Read repartitioned df back into spark

In [22]:
df_fhvhv = spark.read.parquet('fhvhv/2021/01/')

In [23]:
df_fhvhv

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: int]

In [24]:
df_fhvhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [26]:
(df_fhvhv.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')
 .filter(df_fhvhv.hvfhs_license_num == 'HV0003')
 .show())

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-12 09:46:38|2021-01-12 09:59:31|           4|         137|
|2021-01-22 18:07:54|2021-01-22 18:13:25|         242|          78|
|2021-01-19 08:55:37|2021-01-19 09:09:36|         244|         128|
|2021-01-12 04:16:21|2021-01-12 04:37:52|         230|         225|
|2021-01-20 17:00:35|2021-01-20 17:12:54|         228|          26|
|2021-01-29 17:27:39|2021-01-29 18:03:28|         123|         130|
|2021-01-16 14:47:42|2021-01-16 15:06:44|          47|         119|
|2021-01-03 05:33:48|2021-01-03 06:04:01|          14|          75|
|2021-01-10 16:31:19|2021-01-10 16:36:52|         150|         150|
|2021-01-30 17:44:41|2021-01-30 17:52:11|         205|         205|
|2021-01-15 15:15:26|2021-01-15 15:40:27|         129|         145|
|2021-01-14 09:18:22|2021-01-14 09:45:49|       

## Pyspark in-built sql functions

In [35]:
from pyspark.sql import functions as F
from pyspark.sql import types

In [36]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [37]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [39]:
(df_fhvhv
 .withColumn('pickup_date', F.to_date(df_fhvhv.pickup_datetime))
 .withColumn('dropoff_date', F.to_date(df_fhvhv.dropoff_datetime)) 
 .withColumn('base_id', crazy_stuff_udf(df_fhvhv.dispatching_base_num))
 .show()
)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|base_id|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------+
|           HV0005|              B02510|2021-01-25 18:08:34|2021-01-25 18:20:34|         247|         169|   NULL| 2021-01-25|  2021-01-25|  e/9ce|
|           HV0005|              B02510|2021-01-01 06:52:41|2021-01-01 06:59:34|          14|         227|   NULL| 2021-01-01|  2021-01-01|  e/9ce|
|           HV0005|              B02510|2021-01-29 10:47:17|2021-01-29 11:06:12|         134|         138|   NULL| 2021-01-29|  2021-01-29|  e/9ce|
|           HV0003|              B02866|2021-01-12 09:46:38|2021-01-12 09:59:31|           4|         137|   NUL

### Read some yellow csv

In [29]:
yellow_file_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-06.csv.gz'
spark.sparkContext.addFile(yellow_file_url)

24/03/06 07:10:52 WARN SparkContext: The path https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-06.csv.gz has been added already. Overwriting of added paths is not supported in the current version.


In [40]:
df_yellow = spark.read.csv(SparkFiles.get('yellow_tripdata_2019-06.csv.gz'), header=True, inferSchema=True)

In [41]:
df_yellow = df_yellow.withColumn('service_type', F.lit('yellow'))

In [42]:
df_yellow.createOrReplaceTempView('yellow_trips_june_2019')

In [43]:
green_file_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-06.csv.gz'
spark.sparkContext.addFile(green_file_url)

In [44]:
df_green = spark.read.csv(SparkFiles.get('green_tripdata_2019-06.csv.gz'), header=True, inferSchema=True)

In [45]:
df_green = df_green.withColumn('service_type', F.lit('green'))

In [46]:
df_green.createOrReplaceTempView('green_trips_june_2019')

In [47]:
spark.sql("""
select 
    service_type,
    count(1)
from yellow_trips_june_2019
group by service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|      yellow| 6941024|
+------------+--------+



## Group by

In [48]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow_trips_june_2019
GROUP BY
    1, 2
""")

In [49]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green_trips_june_2019
GROUP BY
    1, 2
""")

## Join

In [50]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour','zone'],how='outer')

In [51]:
df_join.show()

+-------------------+----+------+--------------+-----------------+--------------+
|               hour|zone|amount|number_records|           amount|number_records|
+-------------------+----+------+--------------+-----------------+--------------+
|2008-12-31 23:00:00|  93|  NULL|          NULL|            61.42|             1|
|2008-12-31 23:00:00| 170|  NULL|          NULL|70.27000000000001|             3|
|2009-01-01 00:00:00|  90|  NULL|          NULL|             11.3|             1|
|2009-01-01 00:00:00| 138|  NULL|          NULL|             35.9|             1|
|2009-01-01 00:00:00| 238|  NULL|          NULL|             30.6|             2|
|2009-01-01 08:00:00| 231|  NULL|          NULL|             42.8|             1|
|2009-01-01 11:00:00| 237|  NULL|          NULL|              8.3|             1|
|2009-01-01 14:00:00|  68|  NULL|          NULL|              9.8|             1|
|2019-05-30 14:00:00|  79|  NULL|          NULL|            11.16|             1|
|2019-05-30 14:0

Now join with zones dataframe to get information on zone names

In [52]:
df_zones

DataFrame[LocationID: int, Borough: string, Zone: string, service_zone: string]

In [53]:
df_res = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [54]:
df_res.drop('LocationID', 'zone').show()

+-------------------+----+------+--------------+-----------------+--------------+---------+--------------------+------------+
|               hour|zone|amount|number_records|           amount|number_records|  Borough|                Zone|service_zone|
+-------------------+----+------+--------------+-----------------+--------------+---------+--------------------+------------+
|2008-12-31 23:00:00|  93|  NULL|          NULL|            61.42|             1|   Queens|Flushing Meadows-...|   Boro Zone|
|2008-12-31 23:00:00| 170|  NULL|          NULL|70.27000000000001|             3|Manhattan|         Murray Hill| Yellow Zone|
|2009-01-01 00:00:00|  90|  NULL|          NULL|             11.3|             1|Manhattan|            Flatiron| Yellow Zone|
|2009-01-01 00:00:00| 138|  NULL|          NULL|             35.9|             1|   Queens|   LaGuardia Airport|    Airports|
|2009-01-01 00:00:00| 238|  NULL|          NULL|             30.6|             2|Manhattan|Upper West Side N...| Yello